# Fitting a model to data with intrinsic scatter or underestimated uncertainties

_Inspired by [Hogg et al. 2010](https://arxiv.org/abs/1008.4686) and [@jakevdp's notes](https://github.com/jakevdp/ESAC-stats-2014)_.

Python imports we'll need later...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline
rnd = np.random.RandomState(seed=42)

---

## First, the simple case: fitting a straight line to data

### Intro and choice of objective function

I want to start with a problem that everyone is probably familiar with or has at least seen before. The problem is this: we observe $N$ independent data points $\boldsymbol{y}=\{y_1,y_2,...y_N\}$ with uncertainties $\boldsymbol{\sigma}=\{\sigma_1,\sigma_2,...\sigma_N\}$ at perfectly-measured values $\boldsymbol{x}=\{x_1,x_2,...x_N\}$. We have reason to believe that the these data were generated by a process that is well-represented by a straight-line, and the only reason that the data deviate from this straight line is because of uncorrelated, Gaussian measurement noise in the $y$-direction. Let's first generate some data that meet these qualifications:

In [ ]:
n_data = 16 # number of data points
a_true = 1.255 # randomly chosen truth
b_true = 4.507 

In [ ]:
# randomly generate some x values over some domain by sampling from a uniform distribution
x = rnd.uniform(0, 2., n_data)
x.sort() # sort the values in place

# evaluate the true model at the given x values
y = a_true*x + b_true

# Heteroscedastic Gaussian uncertainties only in y direction
y_err = rnd.uniform(0.1, 0.2, size=n_data) # randomly generate uncertainty for each datum
y = rnd.normal(y, y_err) # re-sample y data with noise

In [ ]:
plt.errorbar(x, y, y_err, linestyle='none', marker='o', ecolor='#666666')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.tight_layout()

Now let's forget that we did that -- we know nothing about the model parameters, except that we think the true values of the data are well-described by a linear relation! We would like to measure the "best-fit" parameters of this model (for a straight line, the slope and intercept $(a,b)$) given the data above. In math, our model for the data $y$ is:
$$
\begin{align}
y &= f(x \,;\, a, b) + {\rm noise}\\
f(x \,;\, a, b) &= a\,x + b
\end{align}
$$

For a given set of parameters, $(a,b)$, we can evaluate our model $f(x \,;\, a, b)$ at a given $x$ location to compute the value of $y$ that we would expect in the absence of noise. For example, for the $n$th datum and for a given set of parameter values $(a,b)$:

$$
\tilde{y}_n = f(x_n \,;\, a, b)
$$

Now, we somehow want to search through all possible values of $a,b$ to find the "best" values, given the data, with some definition of "best." When we say this word, we are implying that we want to _optimize_ (find the maximum or minimum) some _objective function_ (a function that takes our data, our model, and returns a quantification of "best", usually as a scalar). Numerically, this scalar objective function can be any function (though you probably want it to be convex) and you will see different choices in practice. You have some leeway in this choice depending on whether your goal is _prediction_, _discovery_, _data compression_, or _discovery_. 

However, for _inference_&mdash;the typical use-case for us as scientists&mdash;you don't have this freedom: one of the conclusions of this talk is going to be that __you have no choice about what "best" means__! Before we get there, though, let's explore what seem like reasonable choices. 

Here are a few desirable features we'd like any objective function to have:

1. For a given set of parameters, we should compare our predicted values to the measured values and base our objective function on the differences
2. The scalar value should be dimensionless (the value of the objective function shouldn't care if we use kilometers vs. parsecs)
3. Data points that have larger errors should contribute less to the objective function (if a datum has a large offset from the predicted value, it shouldn't matter _if_ the datum has a large uncertainty)
4. Convexity

To meet these three criteria, whatever objective function we choose should operate on the (dimensionless) quantities:

$$
\chi_n = \frac{y_n - \tilde{y}_n}{\sigma_n}
$$

i.e. the difference between our predicted values $\tilde{y}$ and the observed $y$ values, weighted by the inverse uncertainties $\sigma$. The uncertainties have the same units as the data, so this is a dimensionless quantity. It also has the nice property that, as we wanted, points with large uncertainties are _downweighted_ relative to points with small uncertainties. Here are some ideas for objective functions based on this scalar:

- __Weighted absolute deviation__: the sum of the absolute values

    $\sum_n^N \, \left|\chi_n\right|$
    
- __Weighted squared deviation__: the sum of the squares

    $\sum_n^N \, \chi_n^2$
    
- __Weighted absolute deviation to some power__ $p$: 

    $\sum_n^N \, \left|\chi_n\right|^p $
    
All of these functions are convex and so we can pass them in to a naive optimizer to find the best parameters. Let's first implement the functions, and then pass them in to the default `scipy.optimize` function minimizer:

In [ ]:
# 'pars' stands for parameters and, in this case, will be a length-2 iterable 
# x, y, and y_err should all be numpy arrays when passed in

def model(pars, x):
    return pars[0]*x + pars[1]

def weighted_absolute_deviation(pars, x, y, y_err):
    chi = (y-model(pars, x)) / y_err
    return np.sum(np.abs(chi))

def weighted_power_deviation(pars, p, x, y, y_err):
    chi = (y-model(pars, x)) / y_err
    return np.sum(chi**p)

def weighted_squared_deviation(pars, x, y, y_err):
    return weighted_power_deviation(pars, 2, x, y, y_err)

For simplicity, let's just compare the two most common -- the absolute deviation and the squared deviation. We can demonstrate that these are convex (over some domain) by computing the objective function values over a grid of parameter values (a grid in $a, b$):

In [ ]:
# make a 256x256 grid of parameter values centered on the true values
a_grid = np.linspace(a_true-2., a_true+2, 256)
b_grid = np.linspace(b_true-2., b_true+2, 256)
a_grid,b_grid = np.meshgrid(a_grid, b_grid)
ab_grid = np.vstack((a_grid.ravel(), b_grid.ravel())).T

In [ ]:
fig,axes = plt.subplots(1, 2, figsize=(9,5.1), sharex=True, sharey=True)

for i,func in enumerate([weighted_absolute_deviation, weighted_squared_deviation]):
    func_vals = np.zeros(ab_grid.shape[0])
    for j,pars in enumerate(ab_grid):
        func_vals[j] = func(pars, x, y, y_err)

    axes[i].pcolormesh(a_grid, b_grid, func_vals.reshape(a_grid.shape), 
                       cmap='Blues', vmin=func_vals.min(), vmax=func_vals.min()+256) # arbitrary scale
    
    axes[i].set_xlabel('$a$')
    
    # plot the truth
    axes[i].plot(a_true, b_true, marker='o', zorder=10, color='#de2d26')
    axes[i].axis('tight')
    axes[i].set_title(func.__name__, fontsize=14)

axes[0].set_ylabel('$b$')

fig.tight_layout()

There are minima in both cases near the true values of the parameters (good), but the gradient of the function is clearly different (the color scales are the same in each panel above). Let's see what happens when we minimize these objective functions to get the best-fit parameter values:

In [ ]:
from scipy.optimize import minimize

In [ ]:
x0 = [1., 1.] # starting guess for the optimizer 
result_abs = minimize(weighted_absolute_deviation, x0=x0, args=(x, y, y_err), method='powell')
result_sq = minimize(weighted_squared_deviation, x0=x0, args=(x, y, y_err), method='powell')

best_pars_abs = result_abs.x
best_pars_sq = result_sq.x

Let's now plot our two best-fit lines over the data:

In [ ]:
plt.errorbar(x, y, y_err, linestyle='none', marker='o', ecolor='#666666')

x_grid = np.linspace(x.min()-0.1, x.max()+0.1, 128)
plt.plot(x_grid, model(best_pars_abs, x_grid), marker='', linestyle='-', label='absolute deviation')
plt.plot(x_grid, model(best_pars_sq, x_grid), marker='', linestyle='-', label='squared deviation')

plt.xlabel('$x$')
plt.ylabel('$y$')

plt.legend(loc='best')
plt.tight_layout()

Well, by eye they both look reasonable! Are we done?! Not quite -- how do we choose between the two?!

In order to pick between these two, or any of the arbitrary objective functions we could have chosen, we have to _justify_ using one function over the others. In what follows, we'll justify optimizing the sum of the squared deviations (so-called "least-squares fitting") by thinking about the problem _probabilistically_, rather than procedurally.

### Least-squares fitting

Let's review the assumptions we made above in generating our data:

1. The data were generated by a straight line
2. Uncorrelated, _known_ Gaussian uncertainties in $y$ cause deviations between the data and predictions
3. The data points are independent
4. The $x$ data are known perfectly, or at least their uncertainties are _far smaller_ than the uncertainties in $y$

First off, these assumptions tell us that for each datum $(x_n, y_n)$ there is some true $y_{n,{\rm true}}$, and because of limitations in our observing process we can't observe the truth, but we know that the values we do observe will be Gaussian (Normal) distributed around the true value. _(Note: This assumption tends to be a good or at least a conservative approximation in practice, but there are certainly more complex situations when, e.g., you have asymmetric uncertainties, or error distributions with large tails!)_. In math:

$$
\begin{align}
p(y \,|\, y_{\rm true}) &= \mathcal{N}(y \,|\, y_{\rm true}, \sigma^2) \\
\mathcal{N}(y \,|\, y_{\rm true}, \sigma^2) &= (2\pi \sigma^2)^{-1/2} \, \exp\left(-\frac{1}{2} \frac{(y-y_{\rm true})^2}{\sigma^2} \right)
\end{align}
$$

This is the likelihood of observing a particular $y$ given the true $y_{\rm true}$. Note that in our model, all of the $y_{\rm true}$'s must lie on a line. It is also interesting that the argument of the normal distribution looks a lot like $\chi^2$!

What about considering two data points, $y_1$ and $y_2$? Now we need to write down the _joint_ probability

$$
p(y_1, y_2 \,|\, y_{1,{\rm true}}, \sigma_1, y_{2,{\rm true}}, \sigma_2)
$$

But, note that in assumption 3 above, we are assuming the data are independent. In that case, the random error in one point does not affect the random error in any other point, so the joint probability can be turned into a product:

$$
p(\{y_n\} \,|\, \{y_{n,{\rm true}}\}, \{\sigma_n\}) = \prod_n^N \, p(y_n \,|\, y_{n,{\rm true}}, \sigma_n)
$$

This is the full expression for the likelihood of the observed data given the true $y$ values. Recall that these true values, according to our assumptions, must lie on a line with some parameters, and we're trying to infer those parameters! We can compute a particular $y_{n,{\rm true}}$ using $x_n$ and a given set of model parameters $a, b$. With that in mind, we can write the likelihood instead as:

$$
p(\{y_n\} \,|\, a, b, \{x_n\}, \{\sigma_n\}) = \prod_n^N \, p(y_n \,|\, a, b, x_n, \sigma_n)
$$

So what are the "best" values of the parameters $a, b$? They are the ones that _maximize_ this likelihood! 

The product on the right of the likelihood is a product over exponentials (well, Gaussians), which can be annoying to deal with. But, maximizing the likelihood is equivalent to maximizing the _log_-likelihood -- so we can get rid of the product and all of those exponentials by taking the log of both sides:

$$
\begin{align}
\ln p(\{y_n\} \,|\, a, b, \{x_n\}, \{\sigma_n\}) &= \sum_n^N \, \ln p(y_n \,|\, a, b, x_n, \sigma_n) \\
&= \sum_n^N \ln \left[(2\pi \sigma_n^2)^{-1/2} \, 
    \exp\left(-\frac{1}{2} \frac{(y-(a\,x_n+b))^2}{\sigma_n^2} \right) \right] \\
&= -\frac{N}{2}\ln(2\pi) 
    - \frac{1}{2} \sum_n^N \left[\frac{(y-(a\,x_n+b))^2}{\sigma_n^2} + \ln{\sigma_n^2} \right]
\end{align}
$$

In this case, the uncertainties are known and constant, so to maximize this expression we only care that (abbreviating the likelihood as $\mathcal{L}$):

$$
\begin{align}
\ln \mathcal{L} &= - \frac{1}{2} \sum_n^N \left[\frac{(y-(a\,x_n+b))^2}{\sigma_n^2}\right] + {\rm const.} \\
&= - \frac{1}{2} \sum_n^N \, \chi_n^2 + {\rm const.} \\
\end{align}
$$

Apparently, _minimizing_ the sum of the weighted squared deviations is equivalent to _maximizing_ the (log) likelihood derived from thinking about the probability of the data! Above, we did that minimization numerically using an iterative solver. That's fine, but (a) it doesn't directly give us the uncertainties on the inferred model parameters, and (b) there is actually an analytic way to solve this problem using linear algebra that is generally _much_ faster!

### Least-squares with matrix calculus

Using linear algebra, we can simplify and generalize a lot of the expressions above. In what follows, all vectors are column vectors and are represented by lower-case bold symbols. Matrices are upper-case bold symbols.


We'll start by writing our model as a matrix equation. To do that, we need a way to, for a given set of parameters, compute the set of predicted $y$'s. This is done by defining the parameter vector, $\boldsymbol{\theta}$, and a matrix typically called the _design matrix_, $\boldsymbol{X}$:

$$
\boldsymbol{\theta} = \begin{bmatrix} b \\ a \end{bmatrix} \quad 
\boldsymbol{X} = \begin{bmatrix} 1 & x_1 \\ 1 & x_2 \\ \vdots & \vdots \\ 1 & x_N \end{bmatrix}
$$

(note the order of the parameters!). With these definitions, the vector of predicted $y$ values is just

$$
\boldsymbol{y}_{\rm pred} = \boldsymbol{X} \, \boldsymbol{\theta}
$$

so the deviation vector between the prediction and the data is just $(\boldsymbol{y}-\boldsymbol{X} \, \boldsymbol{\theta})$ where

$$
\boldsymbol{y} = \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_N \end{bmatrix}
$$

But how do we include the uncertainties? We'll pack the list of uncertainties (variances) into the trace of a 2D, $N \times N$ matrix called the _covariance matrix_. Because we are assuming the uncertainties are independent, the off-diagonal terms are all zero:

$$
\boldsymbol{\Sigma} = \begin{bmatrix} 
\sigma_1^2 & 0 & \dots & 0 \\ 
0 & \sigma_2^2 & \dots & 0 \\ 
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \sigma_N^2 
\end{bmatrix}
$$

With these matrices, we can write the expression for $\chi^2$ (and therefore the log-likelihood) very concisely:

$$
\begin{align}
\chi^2 &= \left(\boldsymbol{y} - \boldsymbol{X}\,\boldsymbol{\theta}\right)^\mathsf{T} \, 
    \boldsymbol{\Sigma}^{-1} \,
    \left(\boldsymbol{y} - \boldsymbol{X}\,\boldsymbol{\theta}\right) \\
\ln\mathcal{L} &= -\frac{1}{2}\left[N\,\ln(2\pi) 
    + \ln|\boldsymbol{\Sigma}|
    + \left(\boldsymbol{y} - \boldsymbol{X}\,\boldsymbol{\theta}\right)^\mathsf{T} \, 
      \boldsymbol{\Sigma}^{-1} \,
      \left(\boldsymbol{y} - \boldsymbol{X}\,\boldsymbol{\theta}\right)
\right]
\end{align}
$$

In [ ]:
a_true = 1.255
b_true = 4.507
V_true = 0.5**2
# V_true = 0.

In [ ]:
n_data = 16

x = rnd.uniform(0, 2., n_data)
y = a_true*x + b_true

idx = x.argsort()
x = x[idx]
y = y[idx]

# uncertainties in y direction
y_err = rnd.uniform(0.1, 0.2, size=n_data)
y = rnd.normal(y, np.sqrt(y_err**2 + V_true))

ivar = 1 / y_err**2 # inverse variance

In [ ]:
A = np.vander(x, N=2)

In [ ]:
ATCinv = (A.T * ivar[None])
ATCinvA = ATCinv.dot(A)

# Note: this is unstable! if cond num is high, could do:
# p,*_ = np.linalg.lstsq(A, y)
p = np.linalg.solve(ATCinvA, ATCinv.dot(y))
p_cov = np.linalg.inv(ATCinvA)

In [ ]:
plt.figure(figsize=(6,6))
plt.errorbar(x, y, y_err, marker='o', linestyle='none')

xlim = plt.xlim()
x_grid = np.linspace(xlim[0], xlim[1], 128)
plt.plot(x_grid, p[0]*x_grid + p[1], marker='', zorder=10, lw=2.)

plt.xlim(xlim)

In [ ]:
eigval,eigvec = np.linalg.eig(p_cov)

In [ ]:
angle = np.degrees(np.arctan2(eigvec[1,0], eigvec[0,0]))
w,h = 2*np.sqrt(eigval)

In [ ]:
from matplotlib.patches import Ellipse

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6,6))

nsigmas = [1, 2]
for n in nsigmas:
    ax.add_patch(Ellipse(p, width=n*w, height=n*h, angle=angle, 
                         fill=False, linewidth=1., edgecolor='k'))
    
ax.plot(a_true, b_true, marker='o', zorder=10, color='#de2d26')

ax.set_xlim(p[0]-0.5, p[0]+0.5)
ax.set_ylim(p[1]-0.5, p[1]+0.5)

ax.set_xlabel('$b$')
ax.set_ylabel('$a$')

---

In [ ]:
def model(p, x):
    return p[0]*x + p[1]

def ln_likelihood(p, x, y, y_ivar):
    dy = y - model(p, x)
    ivar = y_ivar # you'll see why
    return -0.5 * (ivar * dy**2 - np.log(ivar/(2*np.pi)))

def ln_prior(p): 
    a,b = p
    
    lp = 0.
    
    # maybe, because a physical constraint?
    if a < 0 or a > 10.:
        return -np.inf
    else:
        lp += np.log(0.1) # normalization
        
    # technically this is an improper prior on b - we're assuming it's flat / uniform
    #    over a huge range which just introduces a constant offset. 
        
    return lp

def ln_posterior(p, x, y, y_ivar):
    lnp = ln_prior(p)
    if np.isinf(lnp): # short-circuit if the prior is infinite (don't bother computing likelihood)
        return lnp

    lnl = ln_likelihood(p, x, y, y_ivar)
    lnprob = lnp + lnl.sum()

    if np.isnan(lnprob):
        return -np.inf

    return lnprob

In [ ]:
ln_likelihood([a_true, b_true], x, y, ivar).sum(), ln_posterior([a_true, b_true], x, y, ivar)

In [ ]:
a_grid = np.linspace(a_true-1., a_true+1, 256)
b_grid = np.linspace(b_true-1., b_true+1, 256)
a_grid,b_grid = np.meshgrid(a_grid, b_grid)
ab_grid = np.vstack((a_grid.ravel(), b_grid.ravel())).T

In [ ]:
lls = np.zeros(ab_grid.shape[0])
for i,(a,b) in enumerate(ab_grid):
    lls[i] = ln_likelihood([a,b], x, y, ivar).sum()

lls = lls.reshape(a_grid.shape)

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(7.5,6))

cs = ax.pcolormesh(a_grid, b_grid, lls, vmin=lls.max()-50., vmax=lls.max(), cmap='Blues_r')
ax.set_aspect('equal')
cb = fig.colorbar(cs)

nsigmas = range(8)
for n in nsigmas:
    ax.add_patch(Ellipse(p, width=n*w, height=n*h, angle=angle, 
                         fill=False, linewidth=1., edgecolor='#555555'))

ax.plot(a_true, b_true, marker='o', zorder=10, color='#de2d26')

ax.set_xlabel('$b$')
ax.set_ylabel('$a$')

---

In [ ]:
def sample_proposal(a_sigma, b_sigma):
    return np.random.normal(0., [a_sigma, b_sigma])

def metropolis_hastings(p0, n_steps, ln_posterior, ln_post_args=(), proposal_args=()):
    p0 = np.array(p0)
    chain = np.zeros((n_steps, len(p0)))
    ln_probs = np.zeros(n_steps)
    n_accept = 0
    
    ln_probs[0] = ln_posterior(p0, *ln_post_args)
    chain[0] = p0
    for i in range(1,n_steps):
        step = sample_proposal(*proposal_args)
        p = chain[i-1] + step
        
        new_ln_prob = ln_posterior(p, *ln_post_args)
        ln_p_accept = new_ln_prob - ln_probs[i-1]
        if (ln_p_accept > 0) or (ln_p_accept > np.log(np.random.uniform())):
            chain[i] = p
            n_accept += 1
            ln_probs[i] = new_ln_prob
        else:
            chain[i] = chain[i-1]
            ln_probs[i] = ln_probs[i-1]
    
    acc_frac = n_accept / n_steps
    return chain, ln_probs, acc_frac

In [ ]:
p0 = [np.random.uniform(a_true-1., a_true+1.), np.random.uniform(b_true-1., b_true+1.)]
chain,_,acc_frac = metropolis_hastings(p0, 8192, ln_posterior, 
                                       ln_post_args=(x,y,ivar),
                                       proposal_args=(0.05,0.05))
acc_frac

In [ ]:
fig,axes = plt.subplots(len(p0), 1, figsize=(5,7))
for i in range(len(p0)):
    axes[i].plot(chain[:,i], marker='', drawstyle='steps')
axes[0].axhline(a_true, color='r')
axes[1].axhline(b_true, color='r')

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(7.5,6))

cs = ax.pcolormesh(a_grid, b_grid, lls, vmin=lls.max()-50, vmax=lls.max(), cmap='Blues_r')
ax.set_aspect('equal')
cb = fig.colorbar(cs)

ax.plot(a_true, b_true, marker='o', zorder=10, color='#de2d26')
ax.plot(chain[:256,0], chain[:256,1], marker='', color='k', linewidth=1.)

ax.set_xlabel('$b$')
ax.set_ylabel('$a$')